In [1]:
import os
import json

import pandas as pd
import numpy as np

#### 10M prompts dataset

In [2]:
df = pd.read_csv("../data/external/prompts_1k.csv")
df.head().T

,0,1,2,3,4
prompt,A portrait photo of a kangaroo wearing an oran...,inmates with cow heads inside a jailcell,"daguerrotype of a corgi astronaut on the moon,...",totem animal tribal chaman vodoo mask feather ...,p. cubensis
raw_data,"{""image_uri"": ""PENDING"", ""modifiers"": [""portra...","{""image_uri"": ""PENDING"", ""modifiers"": [""inmate...","{""image_uri"": ""PENDING"", ""modifiers"": [""daguer...","{""image_uri"": ""PENDING"", ""modifiers"": [""totem ...","{""image_uri"": ""PENDING"", ""modifiers"": [""p cube..."


In [3]:
all_modifiers_lst = []

for row in df['raw_data']:
    all_modifiers_lst += json.loads(row)['modifiers']

all_modifiers_series = pd.Series(all_modifiers_lst)
counted_mods = all_modifiers_series.value_counts()
counted_mods

artstation                    211
trending                      126
highly detailed               124
greg rutkowski                118
sharp focus                    99
                             ... 
avalon ballroom poster art      1
nebula fractal                  1
rainy day cafe painting         1
irresistible man staring        1
shopping center                 1
Length: 5403, dtype: int64

In [4]:
counted_mods[counted_mods > 2]

artstation         211
trending           126
highly detailed    124
greg rutkowski     118
sharp focus         99
                  ... 
charlie bowater      3
distance             3
pencil drawing       3
realism              3
landscape            3
Length: 440, dtype: int64

In [5]:
df['prompt'][0]

'A portrait photo of a kangaroo wearing an orange hoodie and blue sunglasses standing on the grass in front of the Sydney Opera House holding a sign on the chest that says Welcome Friends, subject: kangaroo, subject detail: wearing orange hoodie, wearing blue sunglasses, subject location: sydney opera house, subject action: holding sign'

#### SD 80k dataset

In [19]:
sd_train_df = pd.read_parquet("../data/external/sd_80k_prompts")
sd_train_df

,Prompt
0,"young, curly haired, redhead Natalie Portman ..."
1,a mystical tribal goddess adorned with feather...
2,"molly millions, portrait of a beautiful cyberp..."
3,"cyborg sweating water, big drops of sweat, for..."
4,"Max Headroom in a Perfume advertisement, magic..."
...,...
81905,ismail inceoglu epic oil on canvas painting of...
81906,eating crayons and being reborn in the loving ...
81907,"ilya kuvshinov with long hair, sky blue hair, ..."
81908,cyberpunk woman with green hair wearing futuri...


In [20]:
sd_split_df = sd_train_df.Prompt.str.split(',', n=3, expand=True)
sd_split_df

,0,1,2,3
0,young,curly haired,redhead Natalie Portman as a optimistic!,"cheerful, giddy medieval innkeeper in a dark ..."
1,a mystical tribal goddess adorned with feather...,full body,gorgeous,"perfect face, powerful, cinematic, beautifull..."
2,molly millions,portrait of a beautiful cyberpunk woman,sunglasses,"shoulder long hair, cyberpunk, street samurai..."
3,cyborg sweating water,big drops of sweat,forehead only,"by Hajime Sorayama, airbrush art, beautiful f..."
4,Max Headroom in a Perfume advertisement,magical,science fiction,"symmetrical face, large eyes, Chanel, Calvin ..."
...,...,...,...,...
81905,ismail inceoglu epic oil on canvas painting of...,line art!!,ink,"art concept for a book cover!!, harmonic colo..."
81906,eating crayons and being reborn in the loving ...,in the style of mondo grosso killian eng kawa...,artstation trending,"8 k, photorealistic, volumetric lighting caus..."
81907,ilya kuvshinov with long hair,sky blue hair,hazel eyes,"boyish face, black leather jacket, profession..."
81908,cyberpunk woman with green hair wearing futuri...,beautiful face,by wlop,"by artgerm, by yoshitaka amano, digital art, ..."


In [21]:
sd_split_df[0].value_counts()

ultra realistic illustration                                         312
concept art by jama jurabaev                                         187
beautiful                                                            129
artgerm                                                              106
digital art                                                           96
                                                                    ... 
hyper realistic hamburger as a xenomorph                               1
A surreal and enigmatic landscape with a spiral galaxy in the sky      1
hyper realistic photo of chaos knight girl                             1
Between heaven and hell                                                1
zulu cyborg                                                            1
Name: 0, Length: 52777, dtype: int64

In [22]:
sd_split_df[2].value_counts()

 elegant                                      2744
 intricate                                    2582
 highly detailed                              1883
 concept art                                  1816
 fantasy                                      1556
                                              ... 
 camomile and dandelion flower messy crown       1
 waves                                           1
 lava                                            1
 android close to camera                         1
 divine tarot card background                    1
Name: 2, Length: 13364, dtype: int64

In [28]:
all_mods_df = pd.Series(','.join(sd_train_df.Prompt).split(','))
all_mods_df = all_mods_df.apply(lambda x: x.strip())
all_mods_df.value_counts()

concept art                                                                                       33451
highly detailed                                                                                   29035
sharp focus                                                                                       27828
artstation                                                                                        25136
digital painting                                                                                  23593
                                                                                                  ...  
a cow with a machine gun                                                                              1
bright rainbow gold cyberpunk nimbus                                                                  1
concept art of modular military base                                                                  1
realistic portrait beautiful detailed 3 d painting of cinematic 

In [42]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='RamAnanth1/distilgpt2-sd-prompts')
set_seed(42)
generator("portrait of a sks young man", max_length=20, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'portrait of a sks young man with red eyes and yellow tounge, long brown hair,'},
 {'generated_text': 'portrait of a sks young man, male, clear face, masculine, upper body, red'},
 {'generated_text': 'portrait of a sks young man holding his fluffy white fluffy blue cat, cloudy sky background lush'},
 {'generated_text': 'portrait of a sks young man with long light brown hair, apron and his blue t'},
 {'generated_text': 'portrait of a sks young man wearing black and gold chains, cinematic lightning, high fantasy,'}]

In [44]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='rexoscare/sd-prompt-generator-gpt-2')
set_seed(42)
generator("portrait of a sks young man", max_length=20, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'portrait of a sks young man, highly detailed, digital painting, artstation, concept art'},
 {'generated_text': 'portrait of a sks young man, 1920s style, smooth, sharp focus, highly detailed'},
 {'generated_text': 'portrait of a sks young man by greg rutkowski, greg rutkowski'},
 {'generated_text': 'portrait of a sks young man wearing a kari dress, holding a bouquet of flowing'},
 {'generated_text': 'portrait of a sks young man wearing front facing!!!! as a princess padme in a pond'}]